In [1]:
# Tensorflow 2.3でメモリを指定及び節約して使うためのおまじない。
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[1], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[1], True)

import numpy as np
import random

np.random.seed(219)
random.seed(219)
tf.random.set_seed(219)

In [2]:
import os
dirpath = "/workdir/taki_lab/tiny-imagenet-200/"
train_dir = os.path.join(dirpath, "train")
val_dir = os.path.join(dirpath, "val")
test_dir = os.path.join(dirpath, "test")

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

N_TRAIN = 100000
N_VAL = 10000
INPUT_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                              rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest")

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.flow_from_directory(train_dir,
                                               target_size=INPUT_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

val_generator = val_gen.flow_from_directory(val_dir,
                                           target_size=INPUT_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical")

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [4]:
from keras import layers, Model

def bottleneck(x, in_ch, out_ch, strides=1):
    params = {
        "padding": "same",
        "kernel_initializer": "he_normal",
        "use_bias": True
    }
    
    inter_ch = out_ch // 4
    h1 = layers.Conv2D(inter_ch, kernel_size=1, strides=strides, **params)(x)
    h1 = layers.BatchNormalization()(h1)
    h1 = layers.ReLU()(h1)
    h1 = layers.Conv2D(inter_ch, kernel_size=3, strides=1, **params)(h1)
    h1 = layers.BatchNormalization()(h1)
    h1 = layers.ReLU()(h1)
    h1 = layers.Conv2D(out_ch, kernel_size=1, strides=1, **params)(h1)
    h1 = layers.BatchNormalization()(h1)
    h1 = layers.ReLU()(h1)
    
    if in_ch != out_ch:
        h2 = layers.Conv2D(out_ch, kernel_size=1, strides=strides, **params)(x)
        h2 = layers.BatchNormalization()(h2)
    else:
        h2 = x
    
    h = layers.Add()([h1, h2])
    h = layers.ReLU()(h)
    return h

def functional_resnet50(input_shape, output_size):
    inputs = layers.Input(input_shape)
    x = layers.Conv2D(64, 7, 2, padding="same", kernel_initializer="he_normal", use_bias=True)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(3, 2, padding="same")(x)
    
    x = bottleneck(x, 64, 256)
    x = bottleneck(x, 256, 256)
    x = bottleneck(x, 256, 256)
    
    x = bottleneck(x, 256, 512, 2)
    x = bottleneck(x, 512, 512)
    x = bottleneck(x, 512, 512)
    x = bottleneck(x, 512, 512)
    
    x = bottleneck(x, 512, 1024, 2)
    x = bottleneck(x,1024, 1024)
    x = bottleneck(x,1024, 1024)
    x = bottleneck(x,1024, 1024)
    x = bottleneck(x,1024, 1024)
    x = bottleneck(x,1024, 1024)
    
    x = bottleneck(x,1024, 2048, 2)
    x = bottleneck(x,2048, 2048)
    x = bottleneck(x,2048, 2048)
    
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(output_size, activation="softmax", kernel_initializer="he_normal")(x)
    return Model(inputs=inputs, outputs=outputs)

In [ ]:
from keras import optimizers
import wandb
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# wandbの設定
wandb.init(project="tiny-image-net", name="resnet50_normal")
config = wandb.config
config.learning_rate = 0.01
config.lr_factor = 0.5
config.optimaizer = "Adam"
config.batch_size = BATCH_SIZE
config.input_size = INPUT_SIZE
config.rotation_range = 10
config.width_shift_range = 0.1
config.height_shift_range = 0.1
config.shear_range = 0.1
config.zoom_range = 0.1

# 最適化設定
model = functional_resnet50((224, 224, 3), 200)
model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
             loss="categorical_crossentropy",
             metrics=["acc"])

early_stopping = EarlyStopping(monitor="val_acc", patience=5)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)

callback_list = [early_stopping, reduce_lr, WandbCallback()]


# 学習
history = model.fit(train_generator,
    steps_per_epoch=N_TRAIN//BATCH_SIZE ,
    epochs=1000,
    validation_data=val_generator,
    validation_steps=N_VAL//BATCH_SIZE,
    use_multiprocessing=True,
    workers=24,
    callbacks=callback_list)

wandb: Wandb version 0.10.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/1000
3125/3125 [==============================] - 686s 220ms/step - loss: 5.1963 - acc: 0.0178 - val_loss: 5.7232 - val_acc: 0.0238
Epoch 2/1000
3125/3125 [==============================] - 682s 218ms/step - loss: 4.4954 - acc: 0.0637 - val_loss: 4.4542 - val_acc: 0.0712
Epoch 3/1000
3125/3125 [==============================] - 683s 219ms/step - loss: 4.1149 - acc: 0.1127 - val_loss: 4.1029 - val_acc: 0.1197
Epoch 4/1000
3125/3125 [==============================] - 685s 219ms/step - loss: 3.8275 - acc: 0.1536 - val_loss: 3.8873 - val_acc: 0.1565
Epoch 5/1000
3125/3125 [==============================] - 684s 219ms/step - loss: 3.5899 - acc: 0.1929 - val_loss: 3.4948 - val_acc: 0.2106
Epoch 6/1000
3125/3125 [==============================] - 688s 220ms/step - loss: 3.3952 - acc: 0.2275 - val_loss: 3.5483 - val_acc: 0.2114
Epoch 7/1000
3125/3125 [==============================] - 689s 220ms/step - loss: 3.2264 - acc: 0.2566 - val_loss: 3.3391 - val_acc: 0.2522
Epoch 8/1000
3125/31